In [1]:
from bs4 import BeautifulSoup
import requests
import os
from dotenv import load_dotenv

In [2]:
# load_dotenv()
# ZENROW_API_KEY = os.getenv("zenrow_scraper_api")

# def search_amazon_and_get_asins(query, limit=4):
#     print(f"🔍 Searching Amazon for: {query}")
#     search_url = "https://api.scraperapi.com/structured/amazon/search"
#     params = {
#         "api_key": ZENROW_API_KEY,
#         "query": query
#     }

#     response = requests.get(search_url, params=params)
#     if response.status_code != 200:
#         print("❌ Failed to fetch search results.")
#         return []

#     data = response.json()
#     if "results" not in data:
#         print("❌ Unexpected response format.")
#         return []

#     asins = []
#     for item in data["results"]:
#         if "asin" in item:
#             asins.append(item["asin"])
#         if len(asins) >= limit:
#             break

#     print(f"✅ Found {len(asins)} ASINs: {asins}")
#     return asins

In [3]:

# def fetch_product_by_asin(asin):
#     print(f"\n📦 Fetching product details for ASIN: {asin}")
#     product_url = "https://api.scraperapi.com/structured/amazon/product"
#     params = {
#         "api_key": SCRAPER_API_KEY,
#         "asin": asin
#     }

#     response = requests.get(product_url, params=params)
#     if response.status_code != 200:
#         print("❌ Failed to fetch product info.")
#         return {}

#     data = response.json()
#     return data


# # 🔄 FULL WORKFLOW
# query = "samsung galaxy s24 ultra"
# asins = search_amazon_and_get_asins(query)

# products = []
# for asin in asins:
#     details = fetch_product_by_asin(asin)
#     if details:
#         products.append({
#             "asin": asin,
#             "name": details.get("name"),
#             "price": details.get("pricing") or details.get("list_price", "Not listed"),
#             "rating": details.get("average_rating"),
#             "image": details["images"][0] if "images" in details and details["images"] else None,
#             "url": f"https://www.amazon.com/dp/{asin}"
#         })

# # ✅ Display Results
# print("\n🎯 Final Structured Product Results:\n")
# for idx, product in enumerate(products, 1):
#     print(f"{idx}. {product['name']}")
#     print(f"   ⭐ Rating: {product['rating']}")
#     print(f"   💲 Price: {product['price']}")
#     print(f"   🔗 URL: {product['url']}")
#     print(f"   🖼️ Image: {product['image']}")
#     print("-" * 80)

In [4]:
# import requests

# SCRAPINGBEE_API = "QSTXXEAL52TEXIMZ8SJNSHWJGF034JNXUVUDUUBDBFR5EGKDQ6RV16ZD1C3FIJN9XC9IZL6PSAECZQM7"
# search_url = "https://www.jumia.com.ng/catalog/?q=samsung+galaxy+s24"

# params = {
#     "api_key": SCRAPINGBEE_API,
#     "url": search_url,
#     "render_js": "true"  # JS rendering is crucial for Jumia
# }

# response = requests.get("https://app.scrapingbee.com/api/v1", params=params)

# # Save result to check if the HTML is good
# with open("jumia_test.html", "w", encoding="utf-8") as f:
#     f.write(response.text)

In [5]:
from dotenv import load_dotenv
import os
import requests
from bs4 import BeautifulSoup

load_dotenv()
SCRAPINGDOG_API_KEY = os.getenv("scraping_dog_api")

def scrape_ajebomarket_with_scrapingdog(query):
    print(f"🔍 Searching Ajebomarket for: {query}")
    base_url = "https://ajebomarket.com/?s="
    search_url = base_url + query.replace(" ", "+")

    params = {
        "api_key": SCRAPINGDOG_API_KEY,
        "url": search_url,
        "dynamic": "true",
    }

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

    response = requests.get("https://api.scrapingdog.com/scrape", params=params, headers=headers)
    print(f"Status: {response.status_code}")
    if response.status_code != 200:
        print(f"❌ Failed with status: {response.status_code}")
        return []

    # Save the HTML to inspect what ScrapingDog returned
    with open("scrapingdog_ajebo_test.html", "w", encoding="utf-8") as f:
        f.write(response.text)

    soup = BeautifulSoup(response.text, "html.parser")
    product_cards = soup.select("div.product__info-wrapper")
    print(f"🧩 Found {len(product_cards)} product cards.")
    products = []

    for card in product_cards:
        try:
            name_elem = card.select_one("h2.h1")
            price_elem = card.select_one("span.price-item--sale, span.price-item--regular")
            image_elem = card.select_one("img")
            link_elem = card.select_one("a.product__title")

            name = name_elem.text.strip() if name_elem else None
            price = price_elem.text.strip() if price_elem else None
            image = image_elem.get("src") if image_elem else None
            if image and image.startswith("//"):
                image = "https:" + image
            link = link_elem.get("href") if link_elem else None
            if link and link.startswith("/"):
                link = "https://ajebomarket.com" + link

            if name and price and image and link:
                products.append({
                    "name": name,
                    "price": price,
                    "image": image,
                    "url": link
                })
        except Exception as e:
            print(f"⚠️ Error parsing product: {e}")
            continue

    print(f"✅ Found {len(products)} products on Ajebomarket")
    return products

In [6]:
# Run test
scrape_ajebomarket_with_scrapingdog("t-shirt")

🔍 Searching Ajebomarket for: t-shirt
Status: 200
🧩 Found 0 product cards.
✅ Found 0 products on Ajebomarket


[]

In [7]:
with open("scrapingdog_ajebo_test.html", "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f.read(), "html.parser")
    print(soup.prettify()[:2000])


<!DOCTYPE html>
<html class="js" lang="en" style="--header-height: 101px;">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <meta content="" name="theme-color"/>
  <link href="https://ajebomarket.com/" rel="canonical"/>
  <link href="//ajebomarket.com/cdn/shop/files/logo_square.png?crop=center&amp;height=32&amp;v=1706642746&amp;width=32" rel="icon" type="image/png"/>
  <link crossorigin="" href="https://fonts.shopifycdn.com" rel="preconnect"/>
  <title>
   Online shopping store for men's clothings and shoes
 – Ajebomarket
  </title>
  <meta content="Onnline and in store shopping for men's shoes, clothings and accessories." name="description"/>
  <meta content="Ajebomarket" property="og:site_name"/>
  <meta content="https://ajebomarket.com/" property="og:url"/>
  <meta content="Online shopping store for men's clothings and shoes" property="og:title"/>
  <meta content="webs

In [8]:
product_cards = soup.select("div.product__info-wrapper")
print(f"🧩 Found {len(product_cards)} product cards.")

for card in product_cards[:3]:  # Preview first 3
    name = card.select_one("h2.h1")
    price = card.select_one("span.price-item--sale, span.price-item--regular")
    print("Name:", name.text.strip() if name else "❌ Not found")
    print("Price:", price.text.strip() if price else "❌ Not found")
    print("---")


🧩 Found 0 product cards.


In [11]:
import os
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv

load_dotenv()
SCRAPINGDOG_API_KEY = os.getenv("scraping_dog_api")

def scrape_ajebomarket_with_scrapingdog(query, save_debug_html=True):
    print(f"🔍 Searching Ajebomarket for: {query}")
    base_url = "https://ajebomarket.com/?s="
    search_url = base_url + query.replace(" ", "+")

    params = {
        "api_key": SCRAPINGDOG_API_KEY,
        "url": search_url,
        "dynamic": "true",
    }

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

    response = requests.get("https://api.scrapingdog.com/scrape", params=params, headers=headers)
    print(f"Status: {response.status_code}")
    if response.status_code != 200:
        print(f"❌ Failed with status: {response.status_code}")
        return []

    # Save raw HTML to debug if needed
    if save_debug_html:
        with open("scrapingdog_ajebo_test.html", "w", encoding="utf-8") as f:
            f.write(response.text)

    soup = BeautifulSoup(response.text, "html.parser")

    # Products are typically rendered inside .product-grid or .card-wrapper
    product_cards = soup.select("div.card-wrapper")
    print(f"🧩 Found {len(product_cards)} product cards.")

    products = []

    for card in product_cards:
        try:
            name_elem = card.select_one("div.card-information h3 a")
            price_elem = card.select_one("span.price")
            image_elem = card.select_one("img")
            link_elem = name_elem

            name = name_elem.text.strip() if name_elem else None
            price = price_elem.text.strip() if price_elem else None
            image = image_elem.get("src") if image_elem else None
            if image and image.startswith("//"):
                image = "https:" + image
            link = link_elem.get("href") if link_elem else None
            if link and link.startswith("/"):
                link = "https://ajebomarket.com" + link

            if name and price and image and link:
                products.append({
                    "name": name,
                    "price": price,
                    "image": image,
                    "url": link
                })

        except Exception as e:
            print(f"⚠️ Error parsing product: {e}")
            continue

    print(f"✅ Found {len(products)} products on Ajebomarket")
    return products

In [12]:
products = scrape_ajebomarket_with_scrapingdog("t-shirt")
for p in products:
    print(p["name"], "-", p["price"])

🔍 Searching Ajebomarket for: t-shirt
Status: 200
🧩 Found 10 product cards.
✅ Found 0 products on Ajebomarket
